In [ ]:
cd /home/drew/NBOH/branchOffMd

In [ ]:
import asyncio
import nest_asyncio
import os

nest_asyncio.apply() #https://github.com/jupyter/notebook/issues/3397#issuecomment-419386811

parent_dir = os.getcwd()
nsimulations = 4
first_sim_n = 9
nproc = 64
proc_per_sim = nproc//nsimulations
stride = 2
START_FRAME = 'N/A'
RESTRAINED_EM_FRAME = 'md_0_1'#'em'
UNRESTRAINED_EM_FRAME = 'em2'#'em2'
RESTRAINT_FRAME = RESTRAINED_EM_FRAME # START_FRAME
NVT_FRAME = 'nvt'
NPT_FRAME = 'npt'
MD_FRAME = 'md_0_2'
RESTRAINED_EM = 'minim_restrained'
UNRESTRAINED_EM = 'minim'
NVT = NVT_FRAME
NPT = NPT_FRAME
MD = 'md'


def grompp(directory, mdp, i, o,  is_continuation, restraint_frame='', topology='topol.top', maxwarn=1):
    parameters = ['gmx', 'grompp', 
                  '-f', f'{mdp}.mdp', 
                  '-c', f'{i}.pdb',  
                  '-p', topology, 
                  '-o', f'{o}.tpr', 
                  '-n',
                  '-maxwarn', f'{maxwarn}']
    if restraint_frame:
        parameters.extend(['-r', f'{restraint_frame}.pdb'])
    if is_continuation:
        parameters.extend(['-t', f'{i}.trr'])
    param_str = " ".join(parameters)
    return f'cd {parent_dir}; cd {directory}; {param_str}'
    #return await asyncio.create_subprocess_shell(cmd, stdout=asyncio.subprocess.DEVNULL, stderr=asyncio.subprocess.DEVNULL)
    #return subp.run(parameters, capture_output=True)

def mdrun(directory, i, nt, gpu_id, pinoffset):
    parameters = ['gmx', 'mdrun', 
                     '-nt', f'{nt}', 
                     '-gpu_id', f'{gpu_id}', 
                     '-ntmpi', '1', 
                     '-pin', 'on', 
                     '-pinstride', f'{stride}', 
                     '-pinoffset', f'{pinoffset}', 
                     '-deffnm', f'{i}', 
                     '-c', f'{i}.pdb', 
                     '-nice', '+10']
    return f'{" ".join(parameters)}'
    #return await asyncio.create_subprocess_shell(cmd, stdout=asyncio.subprocess.DEVNULL, stderr=asyncio.subprocess.DEVNULL)
    #return subp.run(               capture_output=True)

def prep_and_run_phase(directory, mdp, i, o, nt, gpu_id, pinoffset, restraint_frame='', maxwarn=1, is_continuation=False):
    return f'{grompp(directory, mdp, i, o, is_continuation, restraint_frame, maxwarn=maxwarn)}; {mdrun(directory, o, nt, gpu_id, pinoffset)}'
    
def run_sim(sim_dex=0, first_directory=0):
    offset = stride * sim_dex * proc_per_sim
    gpu_id = 0 if sim_dex < nsimulations / 2 else 1 #we only have 2 gpus 0 and 1 otherwise a binary check wouldn't suffice
    directory = f'run{sim_dex + first_directory:02}'
    return ';'.join([
        #prep_and_run_phase(directory, RESTRAINED_EM, START_FRAME, RESTRAINED_EM_FRAME, proc_per_sim, gpu_id, offset, restraint_frame=),
        prep_and_run_phase(directory, UNRESTRAINED_EM, RESTRAINED_EM_FRAME, UNRESTRAINED_EM_FRAME, proc_per_sim, gpu_id, offset),
        prep_and_run_phase(directory, NVT, UNRESTRAINED_EM_FRAME, NVT_FRAME, proc_per_sim, gpu_id, offset, restraint_frame=RESTRAINT_FRAME),
        prep_and_run_phase(directory, NPT, NVT_FRAME, NPT_FRAME, proc_per_sim, gpu_id, offset, restraint_frame=RESTRAINT_FRAME, maxwarn=2, is_continuation=True),
        prep_and_run_phase(directory, MD, NPT_FRAME, MD_FRAME, proc_per_sim, gpu_id, offset, is_continuation=True)])

async def run(cmd, do_stdout=False, do_stderr=False):
    proc = await asyncio.create_subprocess_shell(cmd, stdout=asyncio.subprocess.PIPE, stderr=asyncio.subprocess.PIPE)
    #await proc.wait()
    stdout, stderr = await proc.communicate()
    print(f'[{cmd!r} exited with {proc.returncode}]')
    if do_stdout and stdout:
        print(f'[stdout]\n{stdout.decode()}')
    if do_stderr and stderr:
        print(f'[stderr]\n{stderr.decode()}')

async def main():
    await asyncio.gather(*[run(run_sim(n, first_sim_n), do_stderr=True) for n in range(nsimulations)])
    
asyncio.run(main())
